In [4]:
import os
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import ee
import rs
ee.Initialize()

In [9]:
# This function adds a time band to the image.
def createTimeBand(image):
    # Scale milliseconds by a large constant.
    return image.addBands(image.metadata('system:time_start').divide(1e18))

# This function adds a constant band to the image.


def createConstantBand(image):
    return ee.Image(1).addBands(image)


def addIndices(index_name_list):
    """Add vegetation indices to the image

    Args:
        index_name_list (list): list of indices to add to the image
    
    Returns:
        function: function to add indices to image (for use with map())

        The image must contain bands with literal names:
            'nir', 'red', 'green', 'blue', 'swir1', 'swir2'

        Vegetation indices may include: 
        NDVI = (nir - red) / (nir + red),
        MNDWI = (green - swir1) / (green + swir1),
        EVI = 2.5 * (nir - red) / (nir + 6 * red - 7 * blue + 1),
        GCVI = (nir / green) - 1,
        NBR1 = (nir - swir1) / (nir + swir1), # Also NDMI: https://www.usgs.gov/landsat-missions/normalized-difference-moisture-index
        NBR2 = (nir - swir2) / (nir + swir2),
        NDTI = (swir1 - swir2) / (swir1 + swir2), # for an example: https://www.mdpi.com/2072-4292/8/8/660

        Not included as of now:
        MODCRC = (swir1 - green) / (swir1 + green),
        SAVI = (nir - red) / (nir + red + 0.16),
        STI = swir1 / swir2,
        TVI = 60 * (nir - green) - 100 * (red - green)

    Examples:
        oli8 = prep_oli8_ic(ee_geometry) \
            .map(addIndices(['NDVI', 'MNDWI', 'EVI', 'GCVI']))
    """
    def addIndices_(image):
        if 'NDVI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'red']).rename('NDVI'))

        if 'GCVI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'green']).rename('GCVI'))

        if 'MNDWI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['green', 'swir1']).rename('MNDWI'))
        
        if 'NBR1' in index_name_list: # Also NDMI: https://www.usgs.gov/landsat-missions/normalized-difference-moisture-index
            image = image.addBands(image.normalizedDifference(
                ['nir', 'swir1']).rename('NBR1'))

        if 'NBR2' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'swir2']).rename('NBR2'))
        
        if 'NDTI' in index_name_list: # for an example: https://www.mdpi.com/2072-4292/8/8/660
            image = image.addBands(image.normalizedDifference(
                ['swir1', 'swir2']).rename('NDTI'))
        
        if 'EVI' in index_name_list:
            image = image.addBands(image.expression(
                '2.5 * (im.nir - im.red) / (im.nir + 6 * im.red - 7 * im.blue + 1)', {'im': image}).rename('EVI'))

        if 'GCVI' in index_name_list:
            image = image.addBands(image.expression(
                '(im.nir / im.green) - 1', {'im': image}).rename('GCVI'))

        return image

    return addIndices_


In [10]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
ee_geometry = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))

tm5 = rs.prep_tm5_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
etm7 = rs.prep_etm7_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
oli8 = rs.prep_oli8_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')


# etm7.map(createTimeBand)

landsat = tm5.merge(etm7).merge(oli8) \
    .filter(ee.Filter.lt('CLOUD_COVER', 50)) \
    .map(addIndices(['GCVI']))

landsat_clouds = rs.prep_landsat_clouds(landsat)
landsat_clouds = landsat_clouds.updateMask(landsat_clouds.select('clouds_shadows'))

In [5]:
from math import pi
3*pi
6*pi

18.84955592153876

In [49]:
def addHarmonicBands(start_month):
    """_summary_

    Args:
        start_month_num (int): Month that serves as a zero reference for time

    Returns:
        func: Function that adds harmonic bands to an image for regression

        Returns a function can be used in ee.ImageCollection.map() to add harmonic
        bands to each image in the collection. See Wang et al (2019). 
        
        The harmonic bands are:

        * year_frac: Fraction of a year between 0 and 1, with 0 and 1 being the first day of start month
        * sin(3 * pi * t)
        * cos(3 * pi * t)
        * sin(6 * pi * t)
        * cos(6 * pi * t)


    Examples:
        oli8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_SR') \
            .map(addHarmonicBands(6))
        oli8
            .select(['constant', 'sin3pi', 'cos3pi', 'sin6pi', 'cos6pi', 'B5']) \
            .reduce(ee.Reducer.linearRegression(5, 1))
    """
    # (1e3 * 60 * 60 * 24 * 365) #/ 3.1536e10 # milliseconds in year
    start_month_num = start_month   # e.g., start_month = 6 --> start_date: 2014-0-01

    def addHarmonicBandsFunction(img):
        # get time as a fraction of a year

        img_date = ee.Date(img.get('system:time_start'))
        year = img_date.get('year')
        prev_year = img_date.get('month').subtract(start_month_num).lt(0)
        monsoon_year = ee.Number(year.subtract(prev_year))
        year_start_date = ee.Date.fromYMD(monsoon_year, start_month_num, 1)
        year_frac = img_date.millis().subtract(
            year_start_date.millis()).divide(3.1536e10)
        # img = img.set('year_frac', year_frac)
        img = img.set('year_frac', year_frac)
        year_frac = img.metadata('year_frac')
        time3pi = year_frac.multiply(9.42477796076938)
        time6pi = year_frac.multiply(18.84955592153876)

        # Add harmonic regression bands: sin(3 * pi * t), cos(3 * pi * t),
        # sin(6 * pi * t), cos(6 * pi * t), t = time in years since June 1
        img = img \
            .addBands(year_frac) \
            .addBands(ee.Image(1).rename('constant')) \
            .addBands(time3pi.cos().rename('cos3pi')) \
            .addBands(time3pi.sin().rename('sin3pi')) \
            .addBands(time6pi.cos().rename('cos6pi')) \
            .addBands(time6pi.sin().rename('sin6pi'))
        # img = img.addBands(
        #     img.select('year_frac')
        return img

    return addHarmonicBandsFunction


In [51]:
# harm_func = addRegressionBands(6)

landsat_harmonic_bands = landsat.map(addHarmonicBands(6))

# landsat_harmonic_bands.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'swir2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7981, 6911],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1543515]},
  {'id': 'swir1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7981, 6911],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1543515]},
  {'id': 'nir',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7981, 6911],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1543515]},
  {'id': 'red',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7981, 6911],
   'crs': 'EPSG:32643',
   'crs_transform': [30, 0, 626385, 0, -30, 1543515]},
  {'id': 'green',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [58]:

independent_bands = ee.List(['constant', 'sin3pi', 'cos3pi', 'sin6pi', 'cos6pi'])
dependent_bands = ee.List(['nir', 'swir1', 'swir2', 'GCVI'])

# ee.List([independent_bands, dependent_bands]).getInfo()
                     
landsat_harmonics_regression = landsat_harmonic_bands \
    .select(independent_bands.cat(dependent_bands)) #\
    # .reduce(ee.Reducer.linearRegression(independent_bands.length(), dependent_bands.length()))


# harmonic_coeff = landsat_harmonics_regression \
#     .select('coefficients') \
#     .arrayFlatten(ee.List([independent_bands, dependent_bands]))


In [59]:
landsat_harmonics_regression.first().getInfo()

EEException: Collection.first: Error in map(ID=1_2_LE07_144051_20140611):
Image.select: Pattern 'GCVI' did not match any bands.

In [29]:

# Load the input image collection: projected climate data.
collection = (ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS') 
  .filterDate(ee.Date('2006-01-01'), ee.Date('2099-01-01')) 
  .filter(ee.Filter.eq('scenario', 'rcp85')) 
  .map(createTimeBand) # Map the functions over the collection, to get constant and time bands.
  .map(createConstantBand)
  .select(['constant', 'system:time_start', 'pr_mean', 'tasmax_mean'])) # Select the predictors and the responses.

collection.first().getInfo()
# # Compute ordinary least squares regression coefficients.
# linearRegression = collection.reduce(
#   ee.Reducer.linearRegression({
#     'numX': 2,
#     'numY': 2
# }))

{'type': 'Image',
 'bands': [{'id': 'constant',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'system:time_start',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'pr_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7025, 3105],
   'crs': 'EPSG:4326',
   'crs_transform': [0.00833333333,
    0,
    -125.02083333,
    0,
    0.00833333333,
    24.0625]},
  {'id': 'tasmax_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7025, 3105],
   'crs': 'EPSG:4326',
   'crs_transform': [0.00833333333,
    0,
    -125.02083333,
    0,
    0.00833333333,
    24.0625]}],
 'properties': {'system:index': 'rcp85_200601'}}

In [1]:
collection = ee.ImageCollection('NASA/NEX-DCP30_ENSEMBLE_STATS')#
collection

NameError: name 'ee' is not defined